In [0]:


spark.read.option("basePath","s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/").csv( "s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/2018-08-{04,11,18,25}/ios/sbe_est_app/143441/", sep="\t").createOrReplaceTempView("weekly_data")
spark.sql("select * from weekly_data where _c0=1015763729").show()

# spark.read.option("basePath","s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").csv( "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2010-07-{04,05,06,07,08,09,10}/ios/sbe_est_app/143441/", sep="\t").createOrReplaceTempView("raw_data_daily")
# spark.sql("select *, cast(_c4 as int) as t1, cast(_c7 as int) as _c7 from raw_data_daily where _c5=373998688 order by t1 , _c7 desc   ").show(200)


# monthly data
spark.read.option("basePath","s3://b2c-prod-dca-store-estimates/store_est/v_final/MONTH/").csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/MONTH/2018-08-31/ios/sbe_est_app/143441/", sep="\t").createOrReplaceTempView("monthly_data")
spark.sql("select * from monthly_data where _c0=1015763729").show()

spark.sql('''
SELECT _c1,_c0,_c2, SUM(est) 
FROM
(
    SELECT *, CAST(_c10 as int) AS est FROM weekly_data
)
WHERE _c0=1015763729
GROUP BY est,_c1,_c0  ''').show()

# spark.read.option("basePath","s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/").csv( "s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/2018-08-{05,06,07,08,09,10,11}/ios/sbe_est_app/143441/", sep="\t").createOrReplaceTempView("raw_data_ha")


In [0]:

import datetime
start_week = "2015-01-01"
end_week = "2015-02-01"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start_week.split('-')])
real_date2 = datetime.date(*[int(x) for x in end_week.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(str(real_date1 + datetime.timedelta(days)))

print dates

In [0]:


import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

daily_csv_schema = T.StructType(
    [
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("date", T.StringType(), True),
        T.StructField("platform_id", T.IntegerType(), True),
        T.StructField("vertical", T.IntegerType(), True),
        T.StructField("feed", T.IntegerType(), True),
        T.StructField("id", T.LongType(), True),
        T.StructField("est", T.IntegerType(), True),
        T.StructField("category", T.IntegerType(), True),
        T.StructField("rank", T.IntegerType(), True)
    ]
)

monthly_csv_schema = T.StructType(
    [
        T.StructField("id", T.LongType(), True),
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("category", T.IntegerType(), True),
        T.StructField("year", T.StringType(), True),
        T.StructField("month", T.StringType(), True),
        T.StructField("iphone_free", T.IntegerType(), True),
        T.StructField("iphone_paid", T.IntegerType(), True),
        T.StructField("iphone_revenue", T.IntegerType(), True),
        T.StructField("ipad_free", T.LongType(), True),
        T.StructField("ipad_paid", T.IntegerType(), True),
        T.StructField("ipad_revenue", T.IntegerType(), True)
    ]
)


weekly_csv_schema = T.StructType(
    [
        T.StructField("id", T.LongType(), True),
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("category", T.IntegerType(), True),
        T.StructField("start_date", T.StringType(), True),
        T.StructField("end_date", T.StringType(), True),
        T.StructField("iphone_free", T.IntegerType(), True),
        T.StructField("iphone_paid", T.IntegerType(), True),
        T.StructField("iphone_revenue", T.IntegerType(), True),
        T.StructField("ipad_free", T.LongType(), True),
        T.StructField("ipad_paid", T.IntegerType(), True),
        T.StructField("ipad_revenue", T.IntegerType(), True)
    ]
)
       
        
def check_diff(dates):
    print 'daily_data', dates

    df_1 = spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").schema(daily_csv_schema).csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{%s}/ios/sbe_est_app/*/" % ",".join(dates), sep="\t")
    df_2 = spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/store_est/v_final/MONTH/").schema(monthly_csv_schema).csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/MONTH/{%s}/ios/sbe_est_app/*/" % dates[-1], sep="\t")

    df_1.createOrReplaceTempView("daily_data")
    df_2.createOrReplaceTempView("monthly_data")
    # spark.sql("select * from daily_data").show(2)
    # spark.sql("select * from monthly_data").show(2)

sc.parallelize(map(check_diff, [dates] ), 1)


In [0]:

test_df = spark.sql('''
SELECT * 
FROM   ( 
                SELECT   id, 
                         store_id, 
                         category, 
                         platform_id, 
                         vertical, 
                         feed, 
                         Sum(est) AS est
                FROM     daily_data 
                WHERE    feed IN ( 0, 
                                  1, 
                                  2, 
                                  101, 
                                  100, 
                                  102 ) 
                GROUP BY id, 
                         store_id, 
                         category, 
                         platform_id, 
                         vertical, 
                         feed
                          ) PIVOT ( max(est) FOR feed IN (0, 
                                                          1, 
                                                          2, 
                                                          101, 
                                                          100, 
                                                          102) ) ''').withColumnRenamed("0", platform_feed_to_metric('ios',0)).withColumnRenamed("1", platform_feed_to_metric('ios',1)).withColumnRenamed("2", platform_feed_to_metric('ios',2)).withColumnRenamed("101", platform_feed_to_metric('ios',101)).withColumnRenamed("100", platform_feed_to_metric('ios',100)).withColumnRenamed("102", platform_feed_to_metric('ios',102)).na.fill(0).cache()
test_df.createOrReplaceTempView("transfer")



In [0]:


def platform_feed_to_metric(platform, feed):
    mapping = [
        ['ios', 0, 'iphone_free'],
        ['ios',1, 'iphone_paid'],
        ['ios',2, 'iphone_revenue'],
        ['ios',101,'ipad_free'],
        ['ios',100,'ipad_paid'],
        ['ios',102,'ipad_revenue'],
        ['android',0,'est_free_app_download'],
        ['android',1,'est_paid_app_download'],
        ['android',2,'est_revenue'],
    ]
    return [x for x in mapping if (x[0], x[1]) == (platform, feed)][0][2]
    
platform_feed_to_metric("ios",0)

In [0]:

test_df = spark.sql('''
SELECT * 
FROM   ( 
                SELECT   id, 
                         store_id, 
                         category, 
                         Sum(iphone_free) AS iphone_free,
                         Sum(iphone_paid) AS iphone_paid,
                         Sum(iphone_revenue) AS iphone_revenue,
                         Sum(ipad_free) AS ipad_free,
                         Sum(ipad_paid) AS ipad_paid,
                         Sum(ipad_revenue) AS ipad_revenue

                FROM      
                GROUP BY id, 
                         store_id, 
                         category ) as t ''')
test_df.createOrReplaceTempView("transfer")


In [0]:

# diff_df_1 = spark.sql("select distinct id, store_id from (select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from transfer ) as t1 except all select distinct id, store_id from (select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from monthly_data ) as t2").cache()
# diff_df_2 = spark.sql("select distinct id, store_id from (select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from monthly_data ) as t1 except all select distinct id, store_id from ( select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from transfer ) as t2").cache()


diff_df_1 = spark.sql("select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from transfer except all select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from monthly_data ").cache()
diff_df_2 = spark.sql("select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from monthly_data except all select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from transfer").cache()
print diff_df_1.take(10)
print diff_df_2.take(10)

# print diff_df_1.show()
# print diff_df_2.show()

# diff_df_1.show()
# diff_df_2.show()

In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

start_week = "2015-01-01"
end_week = "2015-02-01"
real_date1 = datetime.date(*[int(x) for x in start_week.split('-')])
real_date2 = datetime.date(*[int(x) for x in end_week.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append(str(real_date1 + datetime.timedelta(days)))

print sar_list
# test_path=list()
# for x in sar_list:
#     for key,item in x.items():
#         test_path.append((key.strftime("%Y-%m-%d"),[d.strftime("%Y-%m-%d") for d in item]))

weekly_csv_schema = T.StructType(
    [
        T.StructField("id", T.LongType(), True),
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("category", T.IntegerType(), True),
        T.StructField("start_date", T.StringType(), True),
        T.StructField("end_date", T.StringType(), True),
        T.StructField("iphone_free", T.IntegerType(), True),
        T.StructField("iphone_paid", T.IntegerType(), True),
        T.StructField("iphone_revenue", T.IntegerType(), True),
        T.StructField("ipad_free", T.LongType(), True),
        T.StructField("ipad_paid", T.IntegerType(), True),
        T.StructField("ipad_revenue", T.IntegerType(), True)
    ]
)
 
df_weekly = spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/").schema(weekly_csv_schema).csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/{%s}/ios/sbe_est_app/*/" % ",".join(sar_list), sep="\t")

df_weekly.createOrReplaceTempView("weekly_data")


In [0]:


# spark.sql("select * from weekly_data where id=378736412 and store_id=143462   ").show()
spark.sql("select * from weekly_data where id=585544408 and store_id=143496 and category=6014 ").show()

In [0]:

spark.sql("select sum(est), feed from daily_data where id=585544408 and store_id=143496 group by feed ").show()
# spark.sql("select * from daily_data where id=378736412 and store_id=143462 and category=100  ").show()



In [0]:

spark.sql("select * from daily_data where id=585544408 and store_id=143496 and feed=101 ").show(20000)


In [0]:

spark.sql("select * from daily_data where id=585544408 and store_id=143496 and category=100 ").show(20000)


In [0]:

spark.sql("select * from weekly_data where id=585544408 and store_id=143496 ").show()

In [0]:

spark.sql("select * from daily_data where id=378736412 and store_id=143462 and category=100 ").show(2)
spark.sql("select * from weekly_data where id=378736412 and store_id=143462 and category=100 ").show()
spark.sql("select * from monthly_data where id=378736412 and store_id=143462 and category=100 ").show()

In [0]:


spark.sql("select * from monthly_data where id=378736412 and store_id=143462 and category=6014 ").show()


In [0]:

spark.sql("select * from daily_data where id=378736412 and store_id=143462 ").show(200)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2010-07-04/ios/sbe_est_app/143449/  --recursive

aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/2010-07-10/ios/sbe_est_app/143449/ --recursive

aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/MONTH/2010-07-31/ios/sbe_est_app/143449/ --recursive



In [0]:
%%sh
